<a href="https://colab.research.google.com/github/kaushal-k-k/CS6910/blob/main/Assignment3/DL_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar


--2021-05-10 11:56:23--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.128, 142.250.31.128, 172.217.15.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  21.6MB/s    in 20s     

2021-05-10 11:56:43 (95.0 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
!tar -xvf  'dakshina_dataset_v1.0.tar'

dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.nonblock.sections.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.omit_pages.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/na

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 100  # Latent dimensionality of the encoding space.
input_token_index = None
target_token_index = None
MAX_LEN_input = None
MAX_LEN_target = None
num_encoder_tokens = 100
num_decoder_tokens = 50
input_tokenizer = None

In [ ]:
def tokenize(data,vocab_size):
  tokenizer = Tokenizer(num_words=vocab_size,char_level=True)
  tokenizer.fit_on_texts(data)
  temp=tokenizer.texts_to_sequences(data)
  # print(data[:3])
  # print(temp[:3])
  dictionary = tokenizer.word_index
  return temp , dictionary , tokenizer

In [ ]:
def load_and_preprocess():
  global input_token_index , target_token_index , MAX_LEN_input , MAX_LEN_target ,num_decoder_tokens,num_encoder_tokens , input_tokenizer
  data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.train.tsv'
  input_texts = []
  target_texts = []
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split('\t')
    input_text, target_text = temp[0],temp[1] 
    target_text = "\t" + target_text + "\n"
    input_text = input_text+"\n"

    input_texts.append(input_text)
    target_texts.append(target_text)
  
  MAX_LEN_input = max([len(txt) for txt in input_texts])
  MAX_LEN_target = max([len(txt) for txt in target_texts])

  # toeknize
  encoder_input , input_token_index , input_tokenizer = tokenize(input_texts , num_encoder_tokens)
  decoder_input , target_token_index,_ = tokenize(target_texts , num_decoder_tokens) 

  # padding
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  decoder_input_data = pad_sequences(decoder_input, maxlen=MAX_LEN_target, dtype='int32', padding='post', truncating='post',value=target_token_index["\n"])

  decoder_target_data = np.zeros((len(input_texts), MAX_LEN_target, num_decoder_tokens), dtype="float32")
  for i,  target_text in enumerate(target_texts):
    for t, char in enumerate(target_text):
      if t > 0:
        decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_target_data[i, t:, target_token_index["\n"]] = 1.0

  return encoder_input_data , decoder_input_data, decoder_target_data

encoder_input_data,decoder_input_data ,decoder_target_data = load_and_preprocess()

In [ ]:
# # Vectorize the data.
# data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.train.tsv'
# input_texts = []
# target_texts = []
# input_characters = set()
# target_characters = set()
# with open(data_path, "r", encoding="utf-8") as f:
#     lines = f.read().split("\n")
# for line in lines[:-1]:
#     temp = line.split('\t')
#     input_text, target_text = temp[0],temp[1]    # We use "tab" as the "start sequence" character
#     # for the targets, and "\n" as "end sequence" character.
#     target_text = "\t" + target_text + "\n"
#     input_text = input_text + "\n"
#     input_texts.append(input_text)
#     target_texts.append(target_text)
#     for char in input_text:
#         if char not in input_characters:
#             input_characters.add(char)
#     for char in target_text:
#         if char not in target_characters:
#             target_characters.add(char)

# input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
# num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])
# print(input_characters)
# print(target_characters)

# print("Number of samples:", len(input_texts))
# print("Number of unique input tokens:", num_encoder_tokens)
# print("Number of unique output tokens:", num_decoder_tokens)
# print("Max sequence length for inputs:", max_encoder_seq_length)
# print("Max sequence length for outputs:", max_decoder_seq_length)

# input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
# target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# encoder_input_data = np.zeros(
#     (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
# )
# decoder_input_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
# )
# decoder_target_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
# )
# print(encoder_input_data.shape)
# for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
#     for t, char in enumerate(input_text):
#         encoder_input_data[i, t, input_token_index[char]] = 1.0
#     encoder_input_data[i, t + 1 :, input_token_index["\n"]] = 1.0
#     for t, char in enumerate(target_text):
#         # decoder_target_data is ahead of decoder_input_data by one timestep
#         decoder_input_data[i, t, target_token_index[char]] = 1.0
#         if t > 0:
#             # decoder_target_data will be ahead by one timestep
#             # and will not include the start character.
#             decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
#     decoder_input_data[i, t + 1 :, target_token_index["\n"]] = 1.0
#     decoder_target_data[i, t:, target_token_index["\n"]] = 1.0

# # print(decoder_input_data[0])
# # print(decoder_target_data[0])

['\n', 'ઁ', 'ં', 'ઃ', 'અ', 'આ', 'ઇ', 'ઈ', 'ઉ', 'ઊ', 'ઋ', 'ઍ', 'એ', 'ઐ', 'ઑ', 'ઓ', 'ઔ', 'ક', 'ખ', 'ગ', 'ઘ', 'ચ', 'છ', 'જ', 'ઝ', 'ઞ', 'ટ', 'ઠ', 'ડ', 'ઢ', 'ણ', 'ત', 'થ', 'દ', 'ધ', 'ન', 'પ', 'ફ', 'બ', 'ભ', 'મ', 'ય', 'ર', 'લ', 'ળ', 'વ', 'શ', 'ષ', 'સ', 'હ', '઼', 'ા', 'િ', 'ી', 'ુ', 'ૂ', 'ૃ', 'ૅ', 'ે', 'ૈ', 'ૉ', 'ો', 'ૌ', '્', 'ૐ']
['\t', '\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Number of samples: 105267
Number of unique input tokens: 65
Number of unique output tokens: 28
Max sequence length for inputs: 19
Max sequence length for outputs: 24
(105267, 19, 65)


In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,Input,GRU,SimpleRNN,Dropout,Embedding,CuDNNLSTM


In [ ]:
def create_model(m_name="LSTM",n_e_layers=1,n_d_layers=1,latent_dim = 100,embedding_size = 16,dropout = 0 , recurrent_dropout = 0):
  keras.backend.clear_session()
  # num_encoder_tokens = 30
  # num_decoder_tokens = 50
  # Define an input sequence and process it.
  input1 = Input(shape=(None,),name= "input_1")
  encoder_inputs = Embedding(input_dim = num_encoder_tokens, output_dim = embedding_size)(input1)

  encoder = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True,return_sequences=True)
  e_o = encoder(encoder_inputs)
  prev = e_o
  for i in range(1,n_e_layers):
    e = globals()[m_name](latent_dim, dropout=dropout,recurrent_dropout = recurrent_dropout,return_state=True,return_sequences=True)
    e_o = e(prev[0])
    prev = e_o
  
  input2 = Input(shape=(None,),name="input_2")
  decoder_inputs = Embedding(input_dim = num_decoder_tokens, output_dim = embedding_size)(input2)
  d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_sequences=True, return_state=True)(decoder_inputs,initial_state = e_o[1:])
  p_d = d_l[0]
  for i in range(1,n_d_layers):
    d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True, return_sequences=True)(p_d,initial_state = e_o[1:])
    p_d = d_l[0]

  
  decoder_dense = Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(d_l[0])

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([input1,input2], decoder_outputs)

  return model


In [ ]:
model = create_model("LSTM",3,2)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 16)     1600        input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, None, 100),  46800       embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [ ]:
def train():
  model = create_model("LSTM",1,2,100,16,0.1,0.1)
  model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
  model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=10,
    validation_split=0.2
  )
  # Save model
  model.save("s2s")
  # print(test_acc("LSTM",100,1,2))
train()

Epoch 1/10
1316/1316 [==============================] - 300s 223ms/step - loss: 1.0337 - accuracy: 0.7224 - val_loss: 1.0099 - val_accuracy: 0.7359
Epoch 2/10
1316/1316 [==============================] - 294s 223ms/step - loss: 0.7074 - accuracy: 0.7813 - val_loss: 0.9305 - val_accuracy: 0.7576
Epoch 3/10
 313/1316 [======>.......................] - ETA: 3:40 - loss: 0.6141 - accuracy: 0.8054

KeyboardInterrupt: ignored

In [ ]:
# model.compile(
#     optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
# )
# model.fit(
#     [encoder_input_data, decoder_input_data],
#     decoder_target_data,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=0.2
# )
# # Save model
# model.save("s2s")


In [ ]:
def load_test_data():
  global input_token_index , MAX_LEN_input , num_encoder_tokens , input_tokenizer
  data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.test.tsv'
  input_texts=[]
  target_texts = []
  # input_characters = set()
  with open(data_path,"r",encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split("\t")
    input_text , target_text = temp[0] ,temp[1]
    input_text = input_text + "\n"
    
    input_texts.append(input_text)
    target_texts.append(target_text)
  
  encoder_input = input_tokenizer.texts_to_sequences(input_texts)
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  print(encoder_input_data.shape)
 
  # for i,input_text in enumerate(input_texts):
  #   for t, char in enumerate(input_text):
  #     if (char ==  'ૠ'):
  #       char = 'ઋ'
  #     encoder_input_data[i, t, input_token_index[char]] = 1.0
  #   encoder_input_data[i, t + 1 :, input_token_index["\n"]] = 1.0

  return encoder_input_data , target_texts
# load_test_data()

In [ ]:
def enc_dec(m_name="LSTM" ,latent_dim = 100, n_e_layers = 1,n_d_layers = 1):
  model = keras.models.load_model("s2s")
 
  if (n_e_layers == 1):
    l_name = ""
  else:
    l_name = "_"+str(n_e_layers-1)

  if (m_name == "SimpleRNN"):
    n_name = "simple_rnn"
  else:
    n_name = m_name

  # encoder
  encoder_inputs = model.input[0]
  encoder_outputs, state_h_enc, state_c_enc = model.get_layer(n_name.lower()+l_name).output  # last encoding layer
  encoder_states = [state_h_enc, state_c_enc]
  encoder_model = keras.Model(encoder_inputs, encoder_states)
  encoder_model.summary()

  # decoder
  decoder_inputs = model.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  d_o = model.get_layer(n_name.lower()+"_"+str(n_e_layers))(decoder_inputs, initial_state=decoder_states_inputs)
  for i in range(1+n_e_layers,n_d_layers+n_e_layers):
    d_o = model.get_layer(n_name.lower()+"_"+str(i))(d_o[0], initial_state=decoder_states_inputs)
  decoder_outputs = d_o[0]
  decoder_states = d_o[1:]
  decoder_dense = model.get_layer("dense")
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
  decoder_model.summary()

  return encoder_model , decoder_model
# enc_dec("LSTM",100,3,2)

ValueError: ignored

In [ ]:

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq,encoder_model,decoder_model):
    global num_decoder_tokens , target_token_index , reverse_target_char_index , max_decoder_seq_length
    # Encode the input as state vectors.

    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [ ]:
def test_acc(m_name="LSTM" ,latent_dim = 100, n_e_layers = 1,n_d_layers = 1):
  x_test , y_test = load_test_data()
  enc ,dec = enc_dec(m_name , latent_dim , n_e_layers ,n_d_layers)
  score = 0 
  print(len(y_test))

  for seq_index in range(len(y_test)):
    
    input_seq = x_test[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,enc,dec)
    if (y_test[seq_index] == decoded_sentence):
      score += 1
    # print("-")
    # print("Input sentence:", y_test[seq_index])
    # print("Decoded sentence:", decoded_sentence)
  print(score/len(y_test))
  return score/len(y_test)

In [ ]:
 print(test_acc("LSTM",100,1,2))

(10373, 19)


ValueError: ignored

In [ ]:
x_test , y_test = load_test_data()
model = keras.models.load_model("s2s")
model.predict(x_test,x_test)


{'\n': 0, 'ઁ': 1, 'ં': 2, 'ઃ': 3, 'અ': 4, 'આ': 5, 'ઇ': 6, 'ઈ': 7, 'ઉ': 8, 'ઊ': 9, 'ઋ': 10, 'ઍ': 11, 'એ': 12, 'ઐ': 13, 'ઑ': 14, 'ઓ': 15, 'ઔ': 16, 'ક': 17, 'ખ': 18, 'ગ': 19, 'ઘ': 20, 'ચ': 21, 'છ': 22, 'જ': 23, 'ઝ': 24, 'ઞ': 25, 'ટ': 26, 'ઠ': 27, 'ડ': 28, 'ઢ': 29, 'ણ': 30, 'ત': 31, 'થ': 32, 'દ': 33, 'ધ': 34, 'ન': 35, 'પ': 36, 'ફ': 37, 'બ': 38, 'ભ': 39, 'મ': 40, 'ય': 41, 'ર': 42, 'લ': 43, 'ળ': 44, 'વ': 45, 'શ': 46, 'ષ': 47, 'સ': 48, 'હ': 49, '઼': 50, 'ા': 51, 'િ': 52, 'ી': 53, 'ુ': 54, 'ૂ': 55, 'ૃ': 56, 'ૅ': 57, 'ે': 58, 'ૈ': 59, 'ૉ': 60, 'ો': 61, 'ૌ': 62, '્': 63, 'ૐ': 64}
['\n', 'ઁ', 'ં', 'ઃ', 'અ', 'આ', 'ઇ', 'ઈ', 'ઉ', 'ઊ', 'ઋ', 'ઍ', 'એ', 'ઐ', 'ઑ', 'ઓ', 'ઔ', 'ક', 'ખ', 'ગ', 'ઘ', 'ચ', 'છ', 'જ', 'ઝ', 'ઞ', 'ટ', 'ઠ', 'ડ', 'ઢ', 'ણ', 'ત', 'થ', 'દ', 'ધ', 'ન', 'પ', 'ફ', 'બ', 'ભ', 'મ', 'ય', 'ર', 'લ', 'ળ', 'વ', 'શ', 'ષ', 'સ', 'હ', '઼', 'ા', 'િ', 'ી', 'ુ', 'ૂ', 'ૃ', 'ૅ', 'ે', 'ૈ', 'ૉ', 'ો', 'ૌ', '્', 'ૐ']
(10373, 19, 65)


ValueError: ignored

In [ ]:
for seq_index in range(20):
    enc ,dec = enc_dec(m_name , latent_dim , n_e_layers ,n_d_layers)
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    print(input_seq)
    decoded_sentence = decode_sequence(input_seq,enc,dec)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
-
Input sentence: અંકાઈ



NameError: ignored